In [1]:
# Goal is to convert x_train from (N x D) to (N x T x D) where T is the number of previous time steps.
# Then apply this N x T x D into RNN/LSTM models

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Train excel sheet

In [3]:
train = pd.read_csv(r"C:\Users\ariha\Desktop\Hackathons & Projects\Wipro Hackathon\train.csv")
train

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175291,2018,12,31,21,30,51,555,168,4,19.4,20.8,1008,91.77,77.86,3.700,204.0,3.5,100
175292,2018,12,31,22,0,37,388,84,4,19.1,20.1,1008,93.88,83.03,3.800,209.0,3.2,100
175293,2018,12,31,22,30,15,115,18,7,19.1,19.6,1008,96.83,88.32,3.800,208.0,2.6,57
175294,2018,12,31,23,0,0,0,0,7,18.7,19.2,1009,96.84,94.34,3.700,206.0,2.1,0


In [4]:
# Creating a Date column
train['Date'] = pd.to_datetime(train[['Year', 'Month', 'Day', 'Hour', 'Minute']])
train.head()

,Year,Month,Day,Hour,Minute,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,2009,1,1,0,0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0,2009-01-01 00:00:00
1,2009,1,1,0,30,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0,2009-01-01 00:30:00
2,2009,1,1,1,0,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0,2009-01-01 01:00:00
3,2009,1,1,1,30,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0,2009-01-01 01:30:00
4,2009,1,1,2,0,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0,2009-01-01 02:00:00


In [5]:
train.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
train.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0,2009-01-01 00:00:00
1,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0,2009-01-01 00:30:00
2,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0,2009-01-01 01:00:00
3,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0,2009-01-01 01:30:00
4,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0,2009-01-01 02:00:00


In [6]:
train.shape

(175296, 14)

In [7]:
columns_list = []

for cols in train.columns:
    columns_list.append(cols)

columns_list

['Clearsky DHI',
 'Clearsky DNI',
 'Clearsky GHI',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag',
 'Date']

In [8]:
columns_list.pop()
columns_list.insert(0, 'Date')
columns_list

['Date',
 'Clearsky DHI',
 'Clearsky DNI',
 'Clearsky GHI',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag']

In [9]:
train = pd.DataFrame(train, columns=columns_list)
train.head()

,Date,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2009-01-01 00:00:00,0,0,0,0,0.0,5.0,1010,75.34,106.15,0.499,346.1,3.1,0
1,2009-01-01 00:30:00,0,0,0,0,1.0,5.0,1010,80.81,112.28,0.490,346.1,3.1,0
2,2009-01-01 01:00:00,0,0,0,4,0.0,5.0,1010,78.27,118.50,0.482,347.9,3.2,0
3,2009-01-01 01:30:00,0,0,0,4,0.0,4.0,1010,78.27,124.78,0.478,347.9,3.1,0
4,2009-01-01 02:00:00,0,0,0,4,0.0,4.0,1010,76.45,131.12,0.475,350.0,3.0,0


In [10]:
corr_matrix = train.corr()
corr_matrix

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
Clearsky DHI,1.000000,0.860988,0.912991,0.022479,0.222931,0.503409,-0.093766,-0.559766,-0.873421,0.207504,-0.000195,0.315655,0.000740
Clearsky DNI,0.860988,1.000000,0.927926,-0.073302,0.033601,0.384353,0.007043,-0.680471,-0.886323,-0.002180,0.006256,0.381162,0.006318
Clearsky GHI,0.912991,0.927926,1.000000,-0.047297,0.140420,0.478273,-0.050556,-0.655765,-0.891033,0.108725,0.004944,0.349180,-0.030297
Cloud Type,0.022479,-0.073302,-0.047297,1.000000,0.202096,0.066777,-0.137158,0.235374,0.026873,0.326904,-0.081565,0.072632,0.090161
Dew Point,0.222931,0.033601,0.140420,0.202096,1.000000,0.848948,-0.419534,0.193455,-0.222336,0.881881,-0.076357,-0.140699,0.036851
Temperature,0.503409,0.384353,0.478273,0.066777,0.848948,1.000000,-0.371395,-0.341914,-0.526548,0.742449,-0.066673,-0.046694,0.015734
Pressure,-0.093766,0.007043,-0.050556,-0.137158,-0.419534,-0.371395,1.000000,-0.085004,0.078900,-0.365342,-0.182006,-0.122674,0.052521
Relative Humidity,-0.559766,-0.680471,-0.655765,0.235374,0.193455,-0.341914,-0.085004,1.000000,0.607108,0.182658,-0.019108,-0.172248,0.023674
Solar Zenith Angle,-0.873421,-0.886323,-0.891033,0.026873,-0.222336,-0.526548,0.078900,0.607108,1.000000,-0.182529,0.003802,-0.295927,-0.029150
Precipitable Water,0.207504,-0.002180,0.108725,0.326904,0.881881,0.742449,-0.365342,0.182658,-0.182529,1.000000,-0.105735,-0.106892,0.044106


In [11]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

C:\Users\ariha\AppData\Local\Temp/ipykernel_17296/2621372733.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
Clearsky DHI,NaN,0.860988,0.912991,0.022479,0.222931,0.503409,-0.093766,-0.559766,-0.873421,0.207504,-0.000195,0.315655,0.000740
Clearsky DNI,NaN,NaN,0.927926,-0.073302,0.033601,0.384353,0.007043,-0.680471,-0.886323,-0.002180,0.006256,0.381162,0.006318
Clearsky GHI,NaN,NaN,NaN,-0.047297,0.140420,0.478273,-0.050556,-0.655765,-0.891033,0.108725,0.004944,0.349180,-0.030297
Cloud Type,NaN,NaN,NaN,NaN,0.202096,0.066777,-0.137158,0.235374,0.026873,0.326904,-0.081565,0.072632,0.090161
Dew Point,NaN,NaN,NaN,NaN,NaN,0.848948,-0.419534,0.193455,-0.222336,0.881881,-0.076357,-0.140699,0.036851
Temperature,NaN,NaN,NaN,NaN,NaN,NaN,-0.371395,-0.341914,-0.526548,0.742449,-0.066673,-0.046694,0.015734
Pressure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.085004,0.078900,-0.365342,-0.182006,-0.122674,0.052521
Relative Humidity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.607108,0.182658,-0.019108,-0.172248,0.023674
Solar Zenith Angle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.182529,0.003802,-0.295927,-0.029150
Precipitable Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.105735,-0.106892,0.044106


In [12]:
columns_to_drop = []
for cols in upper.columns:
    if any (upper[cols] > 0.75):
        columns_to_drop.append(cols)
columns_to_drop

['Clearsky DNI', 'Clearsky GHI', 'Temperature', 'Precipitable Water']

In [13]:
train.drop(columns=['Temperature', 'Precipitable Water'], axis=1, inplace=True)
train.head()

,Date,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,2009-01-01 00:00:00,0,0,0,0,0.0,1010,75.34,106.15,346.1,3.1,0
1,2009-01-01 00:30:00,0,0,0,0,1.0,1010,80.81,112.28,346.1,3.1,0
2,2009-01-01 01:00:00,0,0,0,4,0.0,1010,78.27,118.50,347.9,3.2,0
3,2009-01-01 01:30:00,0,0,0,4,0.0,1010,78.27,124.78,347.9,3.1,0
4,2009-01-01 02:00:00,0,0,0,4,0.0,1010,76.45,131.12,350.0,3.0,0


In [14]:
train.shape

(175296, 12)

In [16]:
y = train.iloc[:, 1:4]
y.head()

,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [19]:
y.shape

(175296, 3)

In [17]:
x = train.iloc[:, 4:]
x.head()

,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,0,0.0,1010,75.34,106.15,346.1,3.1,0
1,0,1.0,1010,80.81,112.28,346.1,3.1,0
2,4,0.0,1010,78.27,118.50,347.9,3.2,0
3,4,0.0,1010,78.27,124.78,347.9,3.1,0
4,4,0.0,1010,76.45,131.12,350.0,3.0,0


In [18]:
x.shape

(175296, 8)

In [20]:
scaler_x = MinMaxScaler((0, 1))
scaler_y = MinMaxScaler((0, 1))

x = scaler_x.fit_transform(x)
y = scaler_y.fit_transform(y)

In [25]:
x.shape[0]

175296

In [24]:
y.shape

(175296, 3)

In [26]:
x_train = []
y_train = []

for i in range(2, x.shape[0]):
    x_train.append(x[i-2 : i])
    y_train.append(y[i])

In [33]:
len(x_train), len(x_train[0]), len(x_train[0][0])

(175294, 2, 8)

In [34]:
len(y_train), len(y_train[0])

(175294, 3)

In [35]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [37]:
x_train.shape, y_train.shape

((175294, 2, 8), (175294, 3))

In [38]:
len(x_train)

175294

In [39]:
x_val = np.array(x_train[int(len(x_train) * 0.90) :])
y_val = np.array(y_train[int(len(y_train) * 0.90) :])

x_val.shape, y_val.shape

((17530, 2, 8), (17530, 3))

In [40]:
x_train = np.array(x_train[: int(len(x_train) * 0.90)])
y_train = np.array(y_train[: int(len(y_train) * 0.90)])

x_train.shape, y_train.shape

((157764, 2, 8), (157764, 3))

# Test Excel sheet

In [57]:
test = pd.read_csv(r"C:\Users\ariha\Desktop\Hackathons & Projects\Wipro Hackathon\test.csv")
test

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Clearsky DHI,Clearsky DNI,Clearsky GHI
0,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,NaN,NaN,NaN
1,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,NaN,NaN,NaN
2,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,NaN,NaN,NaN
3,2019,1,1,1,30,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,NaN,NaN,NaN
4,2019,1,1,2,0,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2019,12,31,21,30,0,5.6,11.1,1006,68.64,77.89,0.8,276,2.2,0,NaN,NaN,NaN
17516,2019,12,31,22,0,0,5.7,9.3,1006,78.31,83.06,0.8,274,1.7,0,NaN,NaN,NaN
17517,2019,12,31,22,30,0,5.7,8.5,1006,82.64,88.34,0.8,273,1.8,0,NaN,NaN,NaN
17518,2019,12,31,23,0,0,4.8,7.8,1006,81.49,94.37,0.8,272,1.8,0,NaN,NaN,NaN


In [58]:
test.drop(test.iloc[:, -3:], axis=1, inplace=True)
test.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0
1,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0
2,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0
3,2019,1,1,1,30,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0
4,2019,1,1,2,0,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0


In [59]:
test['Date'] = pd.to_datetime(test[['Year', 'Month', 'Day', 'Hour', 'Minute']])
test.head()

,Year,Month,Day,Hour,Minute,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,2019,1,1,0,0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,2019-01-01 00:00:00
1,2019,1,1,0,30,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,2019-01-01 00:30:00
2,2019,1,1,1,0,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,2019-01-01 01:00:00
3,2019,1,1,1,30,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,2019-01-01 01:30:00
4,2019,1,1,2,0,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,2019-01-01 02:00:00


In [60]:
test.drop(test.iloc[:, :5], axis=1, inplace=True)
test.head()

,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag,Date
0,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0,2019-01-01 00:00:00
1,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0,2019-01-01 00:30:00
2,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0,2019-01-01 01:00:00
3,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0,2019-01-01 01:30:00
4,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0,2019-01-01 02:00:00


In [61]:
columns_list_test = []

for cols in test.columns:
    columns_list_test.append(cols)
columns_list_test

['Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag',
 'Date']

In [62]:
columns_list_test.pop()
columns_list_test.insert(0, 'Date')

In [63]:
columns_list_test

['Date',
 'Cloud Type',
 'Dew Point',
 'Temperature',
 'Pressure',
 'Relative Humidity',
 'Solar Zenith Angle',
 'Precipitable Water',
 'Wind Direction',
 'Wind Speed',
 'Fill Flag']

In [64]:
test = pd.DataFrame(test, columns=columns_list_test)
test.head()

,Date,Cloud Type,Dew Point,Temperature,Pressure,Relative Humidity,Solar Zenith Angle,Precipitable Water,Wind Direction,Wind Speed,Fill Flag
0,2019-01-01 00:00:00,7,18.4,18.8,1008,97.70,106.23,3.5,190,2.3,0
1,2019-01-01 00:30:00,3,18.4,18.6,1008,98.92,112.36,3.5,187,2.5,0
2,2019-01-01 01:00:00,3,18.2,18.5,1008,98.35,118.58,3.5,184,2.8,0
3,2019-01-01 01:30:00,3,18.2,18.3,1008,99.58,124.86,3.5,185,3.0,0
4,2019-01-01 02:00:00,0,18.0,18.0,1008,99.71,131.20,3.6,186,3.1,0


In [65]:
test.drop(columns=['Temperature', 'Precipitable Water'], axis=1, inplace=True)
test.head()

,Date,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,2019-01-01 00:00:00,7,18.4,1008,97.70,106.23,190,2.3,0
1,2019-01-01 00:30:00,3,18.4,1008,98.92,112.36,187,2.5,0
2,2019-01-01 01:00:00,3,18.2,1008,98.35,118.58,184,2.8,0
3,2019-01-01 01:30:00,3,18.2,1008,99.58,124.86,185,3.0,0
4,2019-01-01 02:00:00,0,18.0,1008,99.71,131.20,186,3.1,0


In [66]:
test.shape

(17520, 9)

In [70]:
x_test_df = test.iloc[:, 1:]
x_test_df.head()

,Cloud Type,Dew Point,Pressure,Relative Humidity,Solar Zenith Angle,Wind Direction,Wind Speed,Fill Flag
0,7,18.4,1008,97.70,106.23,190,2.3,0
1,3,18.4,1008,98.92,112.36,187,2.5,0
2,3,18.2,1008,98.35,118.58,184,2.8,0
3,3,18.2,1008,99.58,124.86,185,3.0,0
4,0,18.0,1008,99.71,131.20,186,3.1,0


In [71]:
x_test_scaled = scaler_x.fit_transform(x_test)
x_test_scaled

array([[0.77777778, 0.7877907 , 0.57142857, ..., 0.52777778, 0.27160494,
        0.        ],
       [0.33333333, 0.7877907 , 0.57142857, ..., 0.51944444, 0.2962963 ,
        0.        ],
       [0.33333333, 0.78197674, 0.57142857, ..., 0.51111111, 0.33333333,
        0.        ],
       ...,
       [0.        , 0.41860465, 0.51428571, ..., 0.75833333, 0.20987654,
        0.        ],
       [0.        , 0.39244186, 0.51428571, ..., 0.75555556, 0.20987654,
        0.        ],
       [0.        , 0.39244186, 0.51428571, ..., 0.75555556, 0.20987654,
        0.        ]])

In [74]:
x_test_scaled.shape[0]

17520

In [76]:
x_test = []

for i in range(2, x_test_scaled.shape[0]):
    x_test.append(x_test_scaled[i-2 : i])

In [78]:
len(x_test), len(x_test[0]), len(x_test[0][0])

(17518, 2, 8)

In [79]:
x_test = np.array(x_test)
x_test.shape

(17518, 2, 8)

# Making sure all arrays are of the right shape

In [80]:
x_train.shape

(157764, 2, 8)

In [81]:
y_train.shape

(157764, 3)

In [82]:
x_val.shape

(17530, 2, 8)

In [83]:
y_val.shape

(17530, 3)

In [84]:
x_test.shape

(17518, 2, 8)